In [0]:
df = spark.read.format("parquet").load("abfss://bronze@datalakeadbete.dfs.core.windows.net/products")

df.display()

In [0]:
df = df.drop("_rescued_data")

df.display()

### **Functions**

In [0]:
df.createOrReplaceTempView("Products_view")

In [0]:
%sql
CREATE OR REPLACE FUNCTION `databricks-catas`.bronze.discount_func(p_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL 
RETURN p_price * 0.90

In [0]:
%sql
SELECT product_id, price, `databricks-catas`.bronze.discount_func(price) AS discounted_price
FROM Products_view

In [0]:
# same way we can do it in pyspark
from pyspark.sql.functions import *
df = df.withColumn("discounted_price", expr("`databricks-catas`.bronze.discount_func(price)"))

df.display()

In [0]:
%sql
CREATE OR REPLACE FUNCTION `databricks-catas`.bronze.upper_func(p_brand STRING)
RETURNS STRING
LANGUAGE PYTHON
as  
$$
  return p_brand.upper()
$$

In [0]:
%sql
select product_id, brand, `databricks-catas`.bronze.upper_func(brand) as brand_upper
from Products_view

### **Data writing**

In [0]:
df.write.format("delta") \
    .mode("overwrite") \
    .save("abfss://silver@datalakeadbete.dfs.core.windows.net/products")

### **Create Schema and Table**

In [0]:
%sql
create table if not exists `databricks-catas`.silver.products_silver
using delta
location "abfss://silver@datalakeadbete.dfs.core.windows.net/products"